In [11]:
'''
Main results table. 
'''
import pandas as pd

df = pd.read_csv('results_avg.csv')
avgdf = df.groupby(['Algorithm', 'Dataset']).mean()
# stddf = df.groupby(['Algorithm', 'Dataset']).std()
avgdf = avgdf.drop(['Iter'], axis=1)
avgdf = avgdf.reset_index()
result = avgdf.pivot('Dataset','Algorithm')
result = result.reset_index()
# change the column dataset to make it easier to read
datasets = ['HYP(0.0001)', 'HYP(0.001)','LED(1)', 'LED(2)', 'RBF(10,0)',
           'RBF(10,0.0001)', 'RBF(10,0.001)', 'RBF(50,0)', 'RBF(50,0.0001)',
           'RBF(50,0.001)', 'SEA(10)', 'SEA(20)', 'airline', 'electricity', 'poker']
result['Dataset'] = datasets
result
result.to_csv('../results/results.csv', float_format='%.3f', index=False)
latex = result.to_latex(float_format='%.2f', index_names=False, 
                index=False, multicolumn=True, multicolumn_format='c',
                bold_rows=True)

latex_list = latex.splitlines()
latex_list.insert(3,'\cmidrule(lr){2-4} \cmidrule(lr){5-7} \cmidrule(lr){8-10} \cmidrule(lr){11-13}')
del(latex_list[4])
latex_list.insert(4, ' & vfdt-{nmin} & cvfdt &  vfdt &  vfdt-{nmin} & cvfdt &   vfdt & vfdt-{nmin} &   cvfdt &   vfdt & vfdt-{nmin} &  cvfdt &  vfdt \\\ ')
# latex_list.insert(4, '{} & vfdt$_{nmin}$ & cvfdt &  vfdt &  vfdt$_{nmin}$ & cvfdt &   vfdt & vfdt$_{nmin}$ &   cvfdt &   vfdt & vfdt$_{nmin}$ &  cvfdt &  vfdt \\\ ')
# latex_list.insert(5,'\midrule')
# del(latex_list[7])
latex_new = '\n'.join(latex_list)

with open('../results/results_tex.tex','w') as text_file:
	print(latex_new, file=text_file)





In [1]:
'''
Table 2: Summarized energy and accuracy values
This script creates the tables for the paper "Hoeffding Trees with nmin adaptation"
Input: Results values: ../results/results.csv
Output: Tables 1: Result with the differences, VFDT vs VFDT-nmin. Table 2: VFDT vs CVFDT
Table 3: No split values. But I need to run the experiments for that! 
VFDT-nmin - VFDT. Negative is a loose in accuracy
Energy: To follow the same approach, negative is good. A negative % means that we decreased the energy by that %
-12%, means we decreased the energy by 12%.
Formula for energy: (-1) * 100 - (VFDT-nmin / VFDT)*100. That shows the % of reduction. Negative: We reduce energy. 
Positive: We save energy
''' 

import pandas as pd

results = pd.read_csv('../results/results.csv')
results = results.drop([0])
# diff_tab = pd.DataFrame(columns = ['Dataset','$\Delta$Acc (\%)', 
#                                    '$\Delta$Energy(\%)','$\Delta$Acc (\%)', 
#                                    '$\Delta$Energy(\%)'],dtype=float) 
diff_tab = pd.DataFrame(columns = ['Dataset','A1', 
                                   'E1','A2', 
                                   'E2'],dtype=float) 
diff_tab['Dataset'] = results['Dataset']

# VFDT
# Accuracy column. Acc nmin - acc vfdt
diff_tab.iloc[:,1] = results.iloc[:,1].astype(float) - results.iloc[:,3].astype(float)
diff_tab.iloc[:,3] = results.iloc[:,1].astype(float) - results.iloc[:,2].astype(float)

# # Energy column. Only checking total energy. 
diff_tab.iloc[:,2] = (100-(results.iloc[:,4].astype(float) / results.iloc[:,6].astype(float))*100)*(-1)
diff_tab.iloc[:,4] = (100-(results.iloc[:,4].astype(float) / results.iloc[:,5].astype(float))*100)*(-1)
# To CSV for plotting purposes
diff_tab.to_csv('../results/table_diff.csv',float_format='%.2f' )
# Average
diff_tab.loc['AVG'] = diff_tab.mean()
diff_tab.at['AVG', 'Dataset'] = 'Average'

# To latex. Format it 
latex = diff_tab.to_latex(float_format='%.2f',index_names=False, index=False, escape=False )
latex_list = latex.splitlines()
latex_list.insert(2,'& \multicolumn{2}{c}{VFDT-nmin vs VFDT }& \multicolumn{2}{c}{VFDT-nmin vs CVFDT} \\\ ')
latex_list.insert(3,'\cmidrule(lr){2-3} \cmidrule(lr){4-5}')
latex_list.insert(4,'Dataset & $\Delta$Acc (\%) & $\Delta$Energy(\%) & $\Delta$Acc (\%) & $\Delta$Energy(\%) \\\ ')
del(latex_list[5])
latex_list.insert(21,'\midrule ')


latex_new = '\n'.join(latex_list)

with open('../results/table_diff.tex','w') as text_file:
	print(latex_new, file=text_file)



,Dataset,A1,E1,A2,E2
12,SEA(20),-0.058000,-7.887026,0.4660,-90.682709
13,airline,-0.142000,-5.194002,12.1640,-83.522726
14,electricity,3.564000,-9.578599,11.9970,-67.562687
15,poker,3.533000,12.207678,16.1800,-77.813680
AVG,AVG,-0.113933,-9.367668,6.3768,-85.657104
